In [ ]:
import requests as r
from io import BytesIO, StringIO
import zipfile
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
base_url = r'http://oasis.caiso.com/oasisapi/SingleZip?'
# ?queryname=PRC_AS&startdatetime=20191018T07:00-0000&enddatetime=20191019T07:00-0000
# &market_run_id=DAM&version=1&as_type=ALL&as_region=ALL

def get_data(params):
    # building URL
    url = base_url
    for key, value in params.items():
        url += f"{key}={value}&"
    data = r.get(url[:-1]).content
    print("URL:", url)
    try:
        filecontent = BytesIO(data)
    except TypeError:
        filecontent = StringIO(data)

    try:
            # have zipfile
        z = zipfile.ZipFile(filecontent)
    except zipfile.BadZipfile:
        return None

        # have unzipped content
    unzipped = [z.read(thisfile) for thisfile in z.namelist()]
    z.close()

        # return
    return unzipped

In [61]:
payload = {
    'queryname': 'PRC_HASP_LMP', 
    'startdatetime': '20190920T07:00-0000',
    'enddatetime': '20191021T07:00-0000',
    'version': '3',
    'market_run_id': 'HASP',
    'grp_type': 'ALL_APNODES'
}
payload = {
    'queryname': 'PRC_HASP_LMP', 
    'startdatetime': '20190920T07:00-0000',
    'enddatetime': '20191021T07:00-0000',
    'version': '3',
    'market_run_id': 'HASP',
    'node': '0096WD_7_N001'
}
payload = {
    'queryname': 'PRC_INTVL_LMP', 
    'startdatetime': '20190919T07:00-0000',
    'enddatetime': '20190920T07:00-0000',
    'version': '1',
    'market_run_id': 'RTM',
    'node': 'LAPLMG1_7_B2'
}

In [62]:
xml = get_data(payload)
display(xml)
xml_to_string = str(xml[0],'utf-8')
root = ET.fromstring(xml_to_string)

URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime=20190919T07:00-0000&enddatetime=20190920T07:00-0000&version=1&market_run_id=RTM&node=LAPLMG1_7_B2&


[b'<?xml version="1.0" encoding="UTF-8"?><OASISReport xmlns="http://www.caiso.com/soa/OASISReport_v1.xsd">\n<MessageHeader>\n<TimeDate>2020-10-27T20:04:01-00:00</TimeDate>\n<Source>OASIS</Source>\n<Version>v20131201</Version>\n</MessageHeader>\n<MessagePayload>\n<RTO>\n<name>CAISO</name>\n<REPORT_ITEM>\n<REPORT_HEADER>\n<SYSTEM>OASIS</SYSTEM>\n<TZ>PPT</TZ>\n<REPORT>PRC_INTVL_LMP</REPORT>\n<MKT_TYPE>RTM</MKT_TYPE>\n<UOM>US$/MWh</UOM>\n<INTERVAL>BEGINNING</INTERVAL>\n<SEC_PER_INTERVAL>300</SEC_PER_INTERVAL>\n</REPORT_HEADER>\n<REPORT_DATA>\n<DATA_ITEM>LMP_CONG_PRC</DATA_ITEM>\n<RESOURCE_NAME>LAPLMG1_7_B2</RESOURCE_NAME>\n<OPR_DATE>2019-09-19</OPR_DATE>\n<INTERVAL_NUM>1</INTERVAL_NUM>\n<INTERVAL_START_GMT>2019-09-19T07:00:00-00:00</INTERVAL_START_GMT>\n<INTERVAL_END_GMT>2019-09-19T07:05:00-00:00</INTERVAL_END_GMT>\n<VALUE>0</VALUE>\n</REPORT_DATA>\n<REPORT_DATA>\n<DATA_ITEM>LMP_CONG_PRC</DATA_ITEM>\n<RESOURCE_NAME>LAPLMG1_7_B2</RESOURCE_NAME>\n<OPR_DATE>2019-09-19</OPR_DATE>\n<INTERVAL_NU

In [64]:
xml_to_string

'<?xml version="1.0" encoding="UTF-8"?><OASISReport xmlns="http://www.caiso.com/soa/OASISReport_v1.xsd">\n<MessageHeader>\n<TimeDate>2020-10-27T20:04:01-00:00</TimeDate>\n<Source>OASIS</Source>\n<Version>v20131201</Version>\n</MessageHeader>\n<MessagePayload>\n<RTO>\n<name>CAISO</name>\n<REPORT_ITEM>\n<REPORT_HEADER>\n<SYSTEM>OASIS</SYSTEM>\n<TZ>PPT</TZ>\n<REPORT>PRC_INTVL_LMP</REPORT>\n<MKT_TYPE>RTM</MKT_TYPE>\n<UOM>US$/MWh</UOM>\n<INTERVAL>BEGINNING</INTERVAL>\n<SEC_PER_INTERVAL>300</SEC_PER_INTERVAL>\n</REPORT_HEADER>\n<REPORT_DATA>\n<DATA_ITEM>LMP_CONG_PRC</DATA_ITEM>\n<RESOURCE_NAME>LAPLMG1_7_B2</RESOURCE_NAME>\n<OPR_DATE>2019-09-19</OPR_DATE>\n<INTERVAL_NUM>1</INTERVAL_NUM>\n<INTERVAL_START_GMT>2019-09-19T07:00:00-00:00</INTERVAL_START_GMT>\n<INTERVAL_END_GMT>2019-09-19T07:05:00-00:00</INTERVAL_END_GMT>\n<VALUE>0</VALUE>\n</REPORT_DATA>\n<REPORT_DATA>\n<DATA_ITEM>LMP_CONG_PRC</DATA_ITEM>\n<RESOURCE_NAME>LAPLMG1_7_B2</RESOURCE_NAME>\n<OPR_DATE>2019-09-19</OPR_DATE>\n<INTERVAL_NUM>

In [67]:
#root[1][0][1][1][]

<Element '{http://www.caiso.com/soa/OASISReport_v1.xsd}REPORT_DATA' at 0x7fce9010e1d8>

In [59]:
# function to convert XML to 
def parse_XML(root):
  df = pd.DataFrame(columns=['Variable', 'Node', 'Date','Value'])
  j = 0
  #
  for i in root[1][0][1][1:]:
    #print(i[2].text, float(i[6].text))\
    new_row = {'Variable':i[0].text, 'Node':i[1].text, 'Date':i[2].text, 'Value':float(i[6].text)}
    #append row to the dataframe
    df = df.append(new_row, ignore_index=True)
  return df

In [60]:
data = parse_XML(root)
data

,Variable,Node,Date,Value
0,LMP_PRC,LAPLMG1_7_B2,2019-09-19,33.79795
1,LMP_PRC,LAPLMG1_7_B2,2019-09-19,26.17796
2,LMP_PRC,LAPLMG1_7_B2,2019-09-19,19.38633
3,LMP_PRC,LAPLMG1_7_B2,2019-09-19,14.77281
4,LMP_PRC,LAPLMG1_7_B2,2019-09-19,24.35226
5,LMP_PRC,LAPLMG1_7_B2,2019-09-19,34.50551
6,LMP_PRC,LAPLMG1_7_B2,2019-09-19,28.33387
7,LMP_PRC,LAPLMG1_7_B2,2019-09-19,45.32648
8,LMP_PRC,LAPLMG1_7_B2,2019-09-19,28.86852
9,LMP_PRC,LAPLMG1_7_B2,2019-09-19,28.03001
